In [1]:
# Deep Convolutional GANs
%reset -f

In [2]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [3]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = (64,64) # We set the size of the generated images (64x64)
num_epochs = 301 # number of epochs to train the DCGAN for
length_of_noise_vector = 10

In [4]:
# Creating the transformations
transform = transforms.Compose([transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images

In [5]:
# Loading the dataset
dataset = dset.ImageFolder(root='input pics for GANs', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.

In [6]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [7]:
# Defining the generator

class G(nn.Module): # We introduce a class to define the generator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the generator.
        super(G, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(length_of_noise_vector, 512, 4, 1, 0, bias = False), # We add an inversed convolution. 512 feature maps. kernel size is 4x4
            nn.BatchNorm2d(512), # We normalize all the features along the dimension of the batch.
            nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # We add another inversed convolution. 256 feature maps. kernel size is 4, stride=2, padding=1
            nn.BatchNorm2d(256), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # We add another inversed convolution. 128 feature maps
            nn.BatchNorm2d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # We add another inversed convolution. 64 feature maps
            nn.BatchNorm2d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # We add another inversed convolution. color images have 3 channels
            nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output containing the generated images.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the generator defined by self.main.
        return output # We return the output containing the generated images.

# Creating the generator
netG = G() # We create the generator object.
netG.apply(weights_init) # We initialize all the weights of its neural network.

G(
  (main): Sequential(
    (0): ConvTranspose2d(10, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [8]:
# Defining the discriminator

class D(nn.Module): # We introduce a class to define the discriminator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the discriminator.
        super(D, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.Conv2d(3, 64, 4, 2, 1, bias = False), # We start with a convolution. color images have 3 channels. 64 feature maps. kernel size is 4x4, stride=2, padding=1
            nn.LeakyReLU(0.2, inplace = True), # We apply a LeakyReLU.
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(128), # We normalize all the features along the dimension of the batch.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(128, 256, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(256, 512, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(512), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), # We add another convolution.
            nn.Sigmoid() # We apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output which will be a value between 0 and 1.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the discriminator defined by self.main.
        return output.view(-1) # We return the output which will be a value between 0 and 1.

# Creating the discriminator
netD = D() # We create the discriminator object.
netD.apply(weights_init) # We initialize all the weights of its neural network.

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [9]:
# Training the DCGANs

criterion = nn.BCELoss() # We create a criterion object that will measure the error between the prediction and the target.
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the discriminator.
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the generator.

for epoch in range(num_epochs): # We iterate over epochs.

    for i, data in enumerate(dataloader, 0): # We iterate over the images of the dataset.
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad() # We initialize to 0 the gradients of the discriminator with respect to the weights.
        
        # Training the discriminator with a real image of the dataset
        real, _ = data # We get a real image of the dataset which will be used to train the discriminator.
        input = Variable(real) # We wrap it in a variable.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(input) # We forward propagate this real image into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_real = criterion(output, target) # We compute the loss between the predictions (output) and the target (equal to 1).
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], length_of_noise_vector, 1, 1)) # We make a random input vector (noise) of the generator.
        fake = netG(noise) # We forward propagate this random input vector into the neural network of the generator to get some fake generated images.
        target = Variable(torch.zeros(input.size()[0])) # We get the target.
        output = netD(fake.detach()) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_fake = criterion(output, target) # We compute the loss between the prediction (output) and the target (equal to 0).

        # Backpropagating the total error
        errD = errD_real + errD_fake # We compute the total error of the discriminator.
        errD.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the discriminator.
        optimizerD.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the discriminator.

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad() # We initialize to 0 the gradients of the generator with respect to the weights.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(fake) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errG = criterion(output, target) # We compute the loss between the prediction (output between 0 and 1) and the target (equal to 1).
        errG.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the generator.
        optimizerG.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the generator.
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, num_epochs, i, len(dataloader), errD.data.item(), errG.data.item())) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        if i%1000==0 and epoch%10==0: # Every 1000 steps and 10 epochs:
            #vutils.save_image(real, '%s/real_samples.png' % "output pics for GANs", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) # We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("output pics for GANs", epoch), normalize = True) # We also save the fake generated images of the minibatch

[0/301][0/8] Loss_D: 1.6567 Loss_G: 3.1920
[0/301][1/8] Loss_D: 2.8554 Loss_G: 4.2840
[0/301][2/8] Loss_D: 1.3190 Loss_G: 6.0122
[0/301][3/8] Loss_D: 0.5371 Loss_G: 6.4904
[0/301][4/8] Loss_D: 0.2944 Loss_G: 6.1497
[0/301][5/8] Loss_D: 0.4829 Loss_G: 6.2067
[0/301][6/8] Loss_D: 0.2690 Loss_G: 6.4676
[0/301][7/8] Loss_D: 0.3183 Loss_G: 8.5185
[1/301][0/8] Loss_D: 0.1244 Loss_G: 6.7655
[1/301][1/8] Loss_D: 0.1459 Loss_G: 6.1242
[1/301][2/8] Loss_D: 0.2784 Loss_G: 6.9780
[1/301][3/8] Loss_D: 0.0928 Loss_G: 6.9305
[1/301][4/8] Loss_D: 0.1925 Loss_G: 6.3364
[1/301][5/8] Loss_D: 0.3258 Loss_G: 6.8616
[1/301][6/8] Loss_D: 0.1609 Loss_G: 7.7586
[1/301][7/8] Loss_D: 0.0430 Loss_G: 7.5404
[2/301][0/8] Loss_D: 0.0922 Loss_G: 7.3310
[2/301][1/8] Loss_D: 0.0993 Loss_G: 7.8713
[2/301][2/8] Loss_D: 0.0799 Loss_G: 7.5748
[2/301][3/8] Loss_D: 0.1334 Loss_G: 7.4985
[2/301][4/8] Loss_D: 0.0832 Loss_G: 7.5133
[2/301][5/8] Loss_D: 0.0794 Loss_G: 7.9070
[2/301][6/8] Loss_D: 0.0796 Loss_G: 8.0489
[2/301][7/8

[23/301][3/8] Loss_D: 2.9614 Loss_G: 12.7356
[23/301][4/8] Loss_D: 2.3796 Loss_G: 6.5827
[23/301][5/8] Loss_D: 0.1614 Loss_G: 3.5483
[23/301][6/8] Loss_D: 1.3261 Loss_G: 11.5704
[23/301][7/8] Loss_D: 2.9028 Loss_G: 4.8181
[24/301][0/8] Loss_D: 0.6967 Loss_G: 7.0381
[24/301][1/8] Loss_D: 0.5383 Loss_G: 4.5982
[24/301][2/8] Loss_D: 1.2374 Loss_G: 9.7517
[24/301][3/8] Loss_D: 1.6973 Loss_G: 2.8993
[24/301][4/8] Loss_D: 2.2829 Loss_G: 12.6615
[24/301][5/8] Loss_D: 1.7228 Loss_G: 8.7533
[24/301][6/8] Loss_D: 0.0928 Loss_G: 4.1826
[24/301][7/8] Loss_D: 0.8166 Loss_G: 8.6662
[25/301][0/8] Loss_D: 0.2644 Loss_G: 6.8829
[25/301][1/8] Loss_D: 0.3512 Loss_G: 3.1881
[25/301][2/8] Loss_D: 1.3938 Loss_G: 11.0187
[25/301][3/8] Loss_D: 2.5142 Loss_G: 4.8814
[25/301][4/8] Loss_D: 0.4418 Loss_G: 4.9104
[25/301][5/8] Loss_D: 0.5711 Loss_G: 8.6024
[25/301][6/8] Loss_D: 0.6598 Loss_G: 5.2133
[25/301][7/8] Loss_D: 0.5043 Loss_G: 5.4923
[26/301][0/8] Loss_D: 0.4204 Loss_G: 7.2728
[26/301][1/8] Loss_D: 0.5686

[46/301][5/8] Loss_D: 1.3201 Loss_G: 0.9762
[46/301][6/8] Loss_D: 1.8026 Loss_G: 7.5510
[46/301][7/8] Loss_D: 1.7382 Loss_G: 3.2130
[47/301][0/8] Loss_D: 0.6688 Loss_G: 4.6313
[47/301][1/8] Loss_D: 0.4573 Loss_G: 3.5055
[47/301][2/8] Loss_D: 0.5092 Loss_G: 3.2576
[47/301][3/8] Loss_D: 0.6347 Loss_G: 4.2549
[47/301][4/8] Loss_D: 0.7104 Loss_G: 1.7998
[47/301][5/8] Loss_D: 1.1471 Loss_G: 7.7910
[47/301][6/8] Loss_D: 2.6654 Loss_G: 1.2896
[47/301][7/8] Loss_D: 1.4518 Loss_G: 8.7115
[48/301][0/8] Loss_D: 1.6560 Loss_G: 1.6448
[48/301][1/8] Loss_D: 1.0734 Loss_G: 5.9298
[48/301][2/8] Loss_D: 0.5668 Loss_G: 3.6805
[48/301][3/8] Loss_D: 0.3513 Loss_G: 3.3790
[48/301][4/8] Loss_D: 0.5532 Loss_G: 3.8059
[48/301][5/8] Loss_D: 0.9141 Loss_G: 1.9893
[48/301][6/8] Loss_D: 0.9451 Loss_G: 6.4794
[48/301][7/8] Loss_D: 1.3062 Loss_G: 1.7704
[49/301][0/8] Loss_D: 1.2292 Loss_G: 6.3170
[49/301][1/8] Loss_D: 1.0365 Loss_G: 2.4784
[49/301][2/8] Loss_D: 0.6891 Loss_G: 4.6786
[49/301][3/8] Loss_D: 0.4715 Los

[70/301][0/8] Loss_D: 1.8160 Loss_G: 1.6116
[70/301][1/8] Loss_D: 1.1031 Loss_G: 2.7815
[70/301][2/8] Loss_D: 0.5426 Loss_G: 3.7647
[70/301][3/8] Loss_D: 0.7647 Loss_G: 1.3793
[70/301][4/8] Loss_D: 0.8995 Loss_G: 3.7934
[70/301][5/8] Loss_D: 0.7606 Loss_G: 1.9822
[70/301][6/8] Loss_D: 0.6709 Loss_G: 2.4248
[70/301][7/8] Loss_D: 0.4227 Loss_G: 2.8248
[71/301][0/8] Loss_D: 0.7781 Loss_G: 3.8854
[71/301][1/8] Loss_D: 0.9234 Loss_G: 1.0546
[71/301][2/8] Loss_D: 0.9833 Loss_G: 4.6356
[71/301][3/8] Loss_D: 1.0686 Loss_G: 1.3496
[71/301][4/8] Loss_D: 0.8356 Loss_G: 4.0333
[71/301][5/8] Loss_D: 0.5779 Loss_G: 2.3802
[71/301][6/8] Loss_D: 0.4663 Loss_G: 2.9837
[71/301][7/8] Loss_D: 0.5380 Loss_G: 3.0867
[72/301][0/8] Loss_D: 0.5009 Loss_G: 2.0890
[72/301][1/8] Loss_D: 0.7769 Loss_G: 3.2473
[72/301][2/8] Loss_D: 0.6724 Loss_G: 1.8484
[72/301][3/8] Loss_D: 0.8094 Loss_G: 3.7362
[72/301][4/8] Loss_D: 0.6540 Loss_G: 1.7555
[72/301][5/8] Loss_D: 0.8142 Loss_G: 3.4523
[72/301][6/8] Loss_D: 0.6188 Los

[93/301][3/8] Loss_D: 0.9086 Loss_G: 4.3234
[93/301][4/8] Loss_D: 0.7527 Loss_G: 1.8251
[93/301][5/8] Loss_D: 0.6307 Loss_G: 3.1629
[93/301][6/8] Loss_D: 0.5757 Loss_G: 2.1402
[93/301][7/8] Loss_D: 0.5944 Loss_G: 2.2783
[94/301][0/8] Loss_D: 0.6727 Loss_G: 3.9897
[94/301][1/8] Loss_D: 0.9804 Loss_G: 1.0134
[94/301][2/8] Loss_D: 0.9423 Loss_G: 3.8493
[94/301][3/8] Loss_D: 0.6090 Loss_G: 2.1980
[94/301][4/8] Loss_D: 0.5396 Loss_G: 2.1275
[94/301][5/8] Loss_D: 0.5753 Loss_G: 3.0476
[94/301][6/8] Loss_D: 0.6369 Loss_G: 1.4253
[94/301][7/8] Loss_D: 0.4281 Loss_G: 4.0566
[95/301][0/8] Loss_D: 0.4801 Loss_G: 3.0760
[95/301][1/8] Loss_D: 0.7048 Loss_G: 1.0747
[95/301][2/8] Loss_D: 0.8068 Loss_G: 4.7144
[95/301][3/8] Loss_D: 1.2493 Loss_G: 0.8213
[95/301][4/8] Loss_D: 1.0234 Loss_G: 4.8345
[95/301][5/8] Loss_D: 1.1709 Loss_G: 0.6423
[95/301][6/8] Loss_D: 1.1713 Loss_G: 5.2010
[95/301][7/8] Loss_D: 1.1447 Loss_G: 0.8456
[96/301][0/8] Loss_D: 1.0413 Loss_G: 4.4505
[96/301][1/8] Loss_D: 0.8397 Los

...

[207/301][7/8] Loss_D: 0.3563 Loss_G: 1.8134
[208/301][0/8] Loss_D: 0.3568 Loss_G: 4.6596
[208/301][1/8] Loss_D: 0.2273 Loss_G: 3.2105
[208/301][2/8] Loss_D: 0.2190 Loss_G: 2.2730
[208/301][3/8] Loss_D: 0.3289 Loss_G: 4.6314
[208/301][4/8] Loss_D: 0.4765 Loss_G: 1.2698
[208/301][5/8] Loss_D: 0.6490 Loss_G: 6.8496
[208/301][6/8] Loss_D: 2.0277 Loss_G: 0.4317
[208/301][7/8] Loss_D: 0.8293 Loss_G: 15.7649
[209/301][0/8] Loss_D: 7.9441 Loss_G: 0.0048
[209/301][1/8] Loss_D: 5.8790 Loss_G: 7.5746
[209/301][2/8] Loss_D: 2.1387 Loss_G: 1.1944
[209/301][3/8] Loss_D: 1.8366 Loss_G: 6.9153
[209/301][4/8] Loss_D: 1.4794 Loss_G: 1.7598
[209/301][5/8] Loss_D: 1.8381 Loss_G: 6.2367
[209/301][6/8] Loss_D: 0.9427 Loss_G: 2.1716
[209/301][7/8] Loss_D: 0.6236 Loss_G: 4.0170
[210/301][0/8] Loss_D: 0.8471 Loss_G: 4.4292
[210/301][1/8] Loss_D: 1.2009 Loss_G: 0.7611
[210/301][2/8] Loss_D: 1.5984 Loss_G: 6.8835
[210/301][3/8] Loss_D: 1.2969 Loss_G: 1.8403
[210/301][4/8] Loss_D: 0.7324 Loss_G: 3.9979
[210/301]

[230/301][6/8] Loss_D: 0.4904 Loss_G: 3.8490
[230/301][7/8] Loss_D: 0.3745 Loss_G: 2.9898
[231/301][0/8] Loss_D: 0.3582 Loss_G: 2.7674
[231/301][1/8] Loss_D: 0.3181 Loss_G: 3.3640
[231/301][2/8] Loss_D: 0.2197 Loss_G: 3.5350
[231/301][3/8] Loss_D: 0.2330 Loss_G: 2.6867
[231/301][4/8] Loss_D: 0.2348 Loss_G: 3.3854
[231/301][5/8] Loss_D: 0.2630 Loss_G: 2.8463
[231/301][6/8] Loss_D: 0.1664 Loss_G: 3.0320
[231/301][7/8] Loss_D: 0.2701 Loss_G: 2.2628
[232/301][0/8] Loss_D: 0.5554 Loss_G: 4.8263
[232/301][1/8] Loss_D: 0.3954 Loss_G: 2.8560
[232/301][2/8] Loss_D: 0.1687 Loss_G: 2.7755
[232/301][3/8] Loss_D: 0.1770 Loss_G: 3.6378
[232/301][4/8] Loss_D: 0.2323 Loss_G: 3.0002
[232/301][5/8] Loss_D: 0.1959 Loss_G: 2.8446
[232/301][6/8] Loss_D: 0.2807 Loss_G: 3.6616
[232/301][7/8] Loss_D: 0.2213 Loss_G: 3.3257
[233/301][0/8] Loss_D: 0.2133 Loss_G: 2.5137
[233/301][1/8] Loss_D: 0.3666 Loss_G: 4.2485
[233/301][2/8] Loss_D: 0.4038 Loss_G: 2.0616
[233/301][3/8] Loss_D: 0.3296 Loss_G: 3.5963
[233/301][

[253/301][5/8] Loss_D: 0.1403 Loss_G: 3.0128
[253/301][6/8] Loss_D: 0.1543 Loss_G: 3.4719
[253/301][7/8] Loss_D: 0.0936 Loss_G: 3.7564
[254/301][0/8] Loss_D: 0.0955 Loss_G: 3.5651
[254/301][1/8] Loss_D: 0.1266 Loss_G: 3.3904
[254/301][2/8] Loss_D: 0.1598 Loss_G: 3.5481
[254/301][3/8] Loss_D: 0.1314 Loss_G: 3.6166
[254/301][4/8] Loss_D: 0.1552 Loss_G: 2.8984
[254/301][5/8] Loss_D: 0.1423 Loss_G: 3.2637
[254/301][6/8] Loss_D: 0.2102 Loss_G: 2.9808
[254/301][7/8] Loss_D: 0.1721 Loss_G: 2.6201
[255/301][0/8] Loss_D: 0.1822 Loss_G: 4.2008
[255/301][1/8] Loss_D: 0.1139 Loss_G: 4.0128
[255/301][2/8] Loss_D: 0.1993 Loss_G: 2.2781
[255/301][3/8] Loss_D: 0.2816 Loss_G: 4.6626
[255/301][4/8] Loss_D: 0.5277 Loss_G: 0.8339
[255/301][5/8] Loss_D: 1.0458 Loss_G: 10.0668
[255/301][6/8] Loss_D: 4.0646 Loss_G: 1.4712
[255/301][7/8] Loss_D: 0.9976 Loss_G: 14.7106
[256/301][0/8] Loss_D: 7.7246 Loss_G: 0.0822
[256/301][1/8] Loss_D: 5.5777 Loss_G: 6.1552
[256/301][2/8] Loss_D: 1.5857 Loss_G: 2.0163
[256/301

[276/301][4/8] Loss_D: 0.1709 Loss_G: 2.9474
[276/301][5/8] Loss_D: 0.1648 Loss_G: 3.1684
[276/301][6/8] Loss_D: 0.1175 Loss_G: 3.8202
[276/301][7/8] Loss_D: 0.2288 Loss_G: 2.4979
[277/301][0/8] Loss_D: 0.2357 Loss_G: 3.6629
[277/301][1/8] Loss_D: 0.2024 Loss_G: 3.4831
[277/301][2/8] Loss_D: 0.1125 Loss_G: 3.7603
[277/301][3/8] Loss_D: 0.1661 Loss_G: 2.9994
[277/301][4/8] Loss_D: 0.1454 Loss_G: 3.4921
[277/301][5/8] Loss_D: 0.0999 Loss_G: 4.1255
[277/301][6/8] Loss_D: 0.2668 Loss_G: 1.7936
[277/301][7/8] Loss_D: 0.0760 Loss_G: 3.2754
[278/301][0/8] Loss_D: 0.4206 Loss_G: 6.2694
[278/301][1/8] Loss_D: 0.6987 Loss_G: 1.5115
[278/301][2/8] Loss_D: 0.4186 Loss_G: 6.2048
[278/301][3/8] Loss_D: 0.7371 Loss_G: 0.3532
[278/301][4/8] Loss_D: 1.8030 Loss_G: 11.0035
[278/301][5/8] Loss_D: 4.8786 Loss_G: 1.9580
[278/301][6/8] Loss_D: 0.9461 Loss_G: 9.0340
[278/301][7/8] Loss_D: 4.6882 Loss_G: 0.0905
[279/301][0/8] Loss_D: 4.2169 Loss_G: 6.9665
[279/301][1/8] Loss_D: 1.6970 Loss_G: 2.3122
[279/301]

[299/301][3/8] Loss_D: 0.1763 Loss_G: 3.5320
[299/301][4/8] Loss_D: 0.1592 Loss_G: 3.6449
[299/301][5/8] Loss_D: 0.1182 Loss_G: 3.6322
[299/301][6/8] Loss_D: 0.1419 Loss_G: 3.0977
[299/301][7/8] Loss_D: 0.1330 Loss_G: 3.4557
[300/301][0/8] Loss_D: 0.1463 Loss_G: 3.2332
[300/301][1/8] Loss_D: 0.1185 Loss_G: 3.6125
[300/301][2/8] Loss_D: 0.1979 Loss_G: 3.1213
[300/301][3/8] Loss_D: 0.0979 Loss_G: 3.8180
[300/301][4/8] Loss_D: 0.1319 Loss_G: 3.3328
[300/301][5/8] Loss_D: 0.1472 Loss_G: 3.0734
[300/301][6/8] Loss_D: 0.1409 Loss_G: 3.0880
[300/301][7/8] Loss_D: 0.2173 Loss_G: 2.2410
